# Проект по SQL

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

# Описание данных
<b>Таблица books</b> - cодержит данные о книгах:
* book_id — идентификатор книги;
* author_id — идентификатор автора;
* title — название книги;
* num_pages — количество страниц;  
* publication_date — дата публикации книги;
* publisher_id — идентификатор издателя.

<b>Таблица authors</b> - cодержит данные об авторах:
* author_id — идентификатор автора;
* author — имя автора.

<b>Таблица publishers</b> -cодержит данные об издательствах:
* publisher_id — идентификатор издательства;
* publisher — название издательства;

<b>Таблица ratings</b> - cодержит данные о пользовательских оценках книг:
* rating_id — идентификатор оценки;
* book_id — идентификатор книги;
* username — имя пользователя, оставившего оценку;
* rating — оценка книги.

<b>Таблица reviews</b> - cодержит данные о пользовательских обзорах:
* review_id — идентификатор обзора;
* book_id — идентификатор книги;
* username — имя автора обзора;
* text — текст обзора.

# Схема данных
!['Схема данных'](https://pictures.s3.yandex.net/resources/scheme_1589269096.png)

# Подключение к базе данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# устанавливаем параметры
# данные удалены, код для примера
db_config = {'user': '', # имя пользователя
             'pwd': '', # пароль
             'host': '',
             'port': , # порт подключения
             'db': ''} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

# Исследование таблиц

In [2]:
# Вывод первых 5 строк всех таблиц
table = ['books', 'authors', 'publishers', 'ratings', 'reviews']
for i in table:
    query = '''
        SELECT *
        FROM {};
    '''.format(i)
    df = pd.io.sql.read_sql(query, con = engine)
    display(df.head())
    display(df.shape)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


(1000, 6)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


(636, 2)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


(340, 2)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


(6456, 4)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


(2793, 4)

# Задания

## Посчитайте, сколько книг вышло после 1 января 2000 года

In [3]:
task1 = '''
SELECT COUNT(*)
FROM books
WHERE CAST(publication_date AS date) > '2000-01-01';
'''
pd.io.sql.read_sql(task1, con = engine) 

,count
0,819


<b>Ответ:</b> 819 книг вышло после 1 января 2000 года

## Для каждой книги посчитайте количество обзоров и среднюю оценку

In [4]:
task2 = '''
            SELECT b.book_id,
                   b.title,
                   COUNT(DISTINCT rv.review_id) AS rewiews_count,
                   AVG(rt.rating) AS avg_rating
            FROM books AS b
            LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
            LEFT JOIN reviews AS rv ON b.book_id = rv.book_id
            GROUP BY b.book_id
            ORDER BY rewiews_count DESC;
        '''
pd.io.sql.read_sql(task2, con = engine)

,book_id,title,rewiews_count,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,963,Water for Elephants,6,3.977273
2,734,The Glass Castle,6,4.206897
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,808,The Natural Way to Draw,0,3.000000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,221,Essential Tales and Poems,0,4.000000


<b>Ответ:</b> Для 1000 книг посчитали  количество озоров и зреднюю оценку. Максимальное количество обзоров у книги Twilight (Twilight #1) и равно 7.

## Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [5]:
task3 = '''
            SELECT p.publisher AS publisher,
                   COUNT(b.book_id) AS books_count
            FROM books AS b
            LEFT JOIN publishers AS p ON b.publisher_id = p.publisher_id
            WHERE b.num_pages > 50
            GROUP BY publisher
            ORDER BY books_count DESC
            LIMIT 1;
        '''
pd.io.sql.read_sql(task3, con = engine)

,publisher,books_count
0,Penguin Books,42


<b>Ответ:</b> Издательство Penguin Books выпустило 42 книги толще 50 страниц - это наибольшее число среди издателей.

## Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [6]:
task4 = '''
            WITH
            rt AS (SELECT book_id,
                          COUNT(rating) AS cnt,
                          AVG(rating) AS avg
                   FROM ratings
                   GROUP BY book_id)
            -----------------------------------       
            SELECT au.author,
                   AVG(rt.avg) AS avg_rt
            FROM  books AS b
            LEFT JOIN rt ON b.book_id = rt.book_id
            LEFT JOIN authors AS au ON b.author_id = au.author_id
            WHERE rt.cnt >= 50
            GROUP BY au.author
            ORDER BY avg_rt DESC
            LIMIT 1;

        '''
pd.io.sql.read_sql(task4, con = engine)

,author,avg_rt
0,J.K. Rowling/Mary GrandPré,4.283844


<b>Ответ:</b> Автор с самой  высокой оценкой книг - J.K. Rowling/Mary GrandPré

## Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

Добавить юзеров с количеством обзоров, посчитать среднее от количества обзоров и сгруппировать по юзерам. Это навскидку, если что, я уже сплю почти так-то)

In [7]:
task5 = ''' 
            SELECT round(AVG(count))
            FROM (SELECT COUNT(review_id)
                  FROM reviews as r
                  WHERE r.username IN (SELECT username
                                       FROM ratings
                                       GROUP BY username
                                       HAVING COUNT(rating) > 50)
                  GROUP BY r.username) AS cnt;
        '''
pd.io.sql.read_sql(task5, con = engine)

,round
0,24.0


<b>Ответ:</b> Среднее количество обзоров от пользователей, которые поставили больше 50 оценок равно 24.